In [1]:
import PyPDF2
from nltk.corpus import stopwords

In [2]:
import nltk
import string
import re
from string import digits
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import CountVectorizer 
from stop_words import get_stop_words

from nltk.stem import PorterStemmer

In [3]:
def preprocessing(path):
    # open PDF
    pdf = PyPDF2.PdfFileReader(open(str(path),"rb"))
    
    # read PDF file in a list
    pdf_content = []
    for page in pdf.pages:
        pdf_content.append(page.extractText())
    pdf_content= str(pdf_content)
        
    #tokenisasi dan remove punctuation
    translator = str.maketrans('','',string.punctuation)
    new_pco = str()
    for i in range(len(pdf_content)):
        if pdf_content[i] == "n":
            if ord(pdf_content[i-1]) == 92:
                continue
            else:new_pco += pdf_content[i]
        elif ord(pdf_content[i]) == 92: continue
        else:new_pco += pdf_content[i]
    pdf_content = new_pco
    pdf_content = pdf_content.translate(translator)
    pdf_content = ''.join([i for i in pdf_content if not i.isdigit()])
    pdf_content = re.sub('r ^ https ?:\\/\\/.*[\\r\\n]*', '', pdf_content, flags=re.MULTILINE)
    pdf_content = word_tokenize(pdf_content)
    print(pdf_content)
        
    #case folding
    caseFolding = [kata.lower() for kata in pdf_content]
    
    #remove stop word
    stopword_list = list(stopwords.words("indonesian"))
    without_stop_words = []
    for word in filter(None, caseFolding):
        if word not in stopword_list:
            without_stop_words.append(word)
            
    #stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    kata_dasar=[]
    for kata in without_stop_words:
        kata_dasar.append(stemmer.stem(kata))

    print(kata_dasar)
    return kata_dasar

In [4]:
from math import log

'''
Using the following formula to calculate BM25
((k3 + 1)q)/((k3 + q)) * ((k1 + 1)f)/((K + f)) * log((r + 0.5)(N − n − R + r + 0.5))/((n − r + 0.5)(R − r + 0.5))
REFERENCE: https://xapian.org/docs/bm25.html
'''

# DEFINING CONSTANTS

k1 = 1.2
b = 0.75
k2 = 100
R = 0 #Since no relevance info is available

# MAIN METHOD
def BM25(docLen, avDocLen, n, N, f, q, r):
    p1 = ((k2 + 1) * q) / (k2 + q)
    p2 = ((k1 + 1) * f) / (getK(docLen, avDocLen) + f)
    p3 = log((r + 0.5) * (N - n - R + r + 0.5)) / ((n - r + 0.5) * (R - r + 0.5))
    return p1 * p2 * p3

def getK(docLen, avDocLen):
    return k1 * ((1 - b) + b * (float(docLen) / float(avDocLen)))

In [5]:
import json
import operator
from collections import defaultdict
#from retrieval import BM25

# get average document length
def get_avdl(length_index):
    corpus_length = 0
    for document in length_index:
        corpus_length += length_index[document]
    return float(corpus_length) / float(len(length_index))

def search(query):
    inv_index_file = open("E:/KULIAH/SEMESTER 8/INRE/Document-Search-Information-Retrieval-BM25-master/dependency/indexes/inverted_index.json","r")
    inverted_index = json.load(inv_index_file)

    length_index_file = open("E:/KULIAH/SEMESTER 8/INRE/Document-Search-Information-Retrieval-BM25-master/dependency/indexes/length_index.json","r")
    length_index = json.load(length_index_file)

    scores = defaultdict(list)
    query_tokens = query.split()
    for token in query_tokens:
        for entry in inverted_index[token]:
            scores[entry[0]] = BM25(length_index[entry[0]],get_avdl(length_index),len(inverted_index[token]),len(length_index),entry[1],1,0)
    return sorted(scores.items(),key=operator.itemgetter(1))

In [6]:
from collections import defaultdict
import glob
import json


def get_file_names():
    files = []
    for file in glob.glob("E:/KULIAH/SEMESTER 8/INRE/Document-Search-Information-Retrieval-BM25-master/dependency/documents/*.pdf"):
        files.append(file)
    return files


def make_index(tokens, document_name, index, length):
    for term in set(tokens):
        index[term].append([document_name,tokens.count(term)])
        length[document_name] = len(set(tokens))


def generator():
    files = get_file_names()
    inverted_index = defaultdict(list)
    length_index = defaultdict(list)
    for file in files:
        make_index(preprocessing(file), file, inverted_index, length_index)
    write(inverted_index,length_index)
    print ("Indexes generated")


def write(inverted_index,length_index):
    inv_index_file = open("E:/KULIAH/SEMESTER 8/INRE/Document-Search-Information-Retrieval-BM25-master/dependency/indexes/inverted_index.json","w")
    json.dump(inverted_index,inv_index_file)

    length_index_file = open("E:/KULIAH/SEMESTER 8/INRE/Document-Search-Information-Retrieval-BM25-master/dependency/indexes/inverted_index.json","w")
    json.dump(length_index,length_index_file)

In [8]:
generator()
option = ""
while option != "q":
    print ("\n")
    print ("Enter search query")
    keywords = input(":: ")
    results = search(keywords)
    print ("\nThe Matching Files Are:")
    for result in results:
        print (result[0])

['Juwairiah', 'ST', 'MPd', 'Alat', 'Peraga', 'dan', 'Media', 'Volume', 'IV', 'Nomor', 'Januari', 'Juni', 'ALAT', 'PERAGA', 'DAN', 'MEDIA', 'PEMBELAJARAN', 'KIMIA', 'Oleh', 'Juwairiah', 'ST', 'MPd', 'Dosen', 'Program', 'Studi', 'Pendidikan', 'Matematika', 'STKIP', 'Bina', 'Bangsa', 'Meulaboh', 'ABSTRAK', 'Alat', 'peraga', 'dan', 'Media', 'pembelajaran', 'kimia', 'memiliki', 'peran', 'dan', 'fungsi', 'yang', 'strategis', 'dalam', 'pencapaian', 'tujuan', 'pembelajaran', 'Penggunaan', 'alat', 'peraga', 'dan', 'media', 'pembelajaran', 'yang', 'bervariasi', 'akan', 'meningkatkan', 'motivasi', 'belajar', 'siswa', 'Kesulitan', 'dalam', 'memahami', 'ilmu', 'kimia', 'disebabkan', 'kebanyakan', 'konsepkonsep', 'dalam', 'ilmu', 'kimia', 'maupun', 'materi', 'kimia', 'secara', 'keseluruhan', 'merupakan', 'konsep', 'atau', 'materi', 'yang', 'bersifat', 'abstrak', 'dan', 'kompleks', 'maka', 'guru', 'harus', 'membantu', 'siswa', 'untuk', 'memvisualisasikan', 'konsep', 'yang', 'abstrak', 'tersebut', 'me

['juwairiah', 'st', 'mpd', 'alat', 'aga', 'media', 'volume', 'iv', 'nomor', 'januari', 'juni', 'alat', 'aga', 'media', 'ajar', 'kimia', 'juwairiah', 'st', 'mpd', 'dosen', 'program', 'studi', 'didik', 'matematika', 'stkip', 'bina', 'bangsa', 'meulaboh', 'abstrak', 'alat', 'aga', 'media', 'ajar', 'kimia', 'milik', 'peran', 'fungsi', 'strategis', 'capai', 'tuju', 'ajar', 'guna', 'alat', 'aga', 'media', 'ajar', 'variasi', 'tingkat', 'motivasi', 'ajar', 'siswa', 'sulit', 'paham', 'ilmu', 'kimia', 'sebab', 'banyak', 'konsepkonsep', 'ilmu', 'kimia', 'materi', 'kimia', 'konsep', 'materi', 'sifat', 'abstrak', 'kompleks', 'guru', 'bantu', 'siswa', 'visualisasi', 'konsep', 'abstrak', 'nyata', 'mudah', 'paham', 'siswa', 'dasar', 'siswa', 'ajar', 'konkret', 'siswa', 'orang', 'dewasa', 'butuh', 'visualisasi', 'paham', 'konsepkonsep', 'abstrak', 'salah', 'upaya', 'alat', 'aga', 'media', 'ajar', 'kimia', 'tanam', 'konsep', 'mudah', 'erti', 'siswa', 'kunci', 'alat', 'aga', 'media', 'ajar', 'kimia', 'ju

['Edisi', 'Juli', 'Volume', 'IX', 'No', 'ISSN', 'ANALISA', 'POLA', 'ASOSIASI', 'JALUR', 'MASUK', 'TERHADAP', 'KELULUSAN', 'MAHASISWA', 'DENGAN', 'MENGGUNAKAN', 'METODE', 'FOLDGROWTH', 'STUDI', 'KASUS', 'FAKULTAS', 'SAINS', 'DAN', 'TEKNOLOGI', 'Mohamad', 'Irfan', 'Jurusan', 'Teknik', 'Informatika', 'Fakultas', 'Sains', 'dan', 'Teknologi', 'Universitas', 'Islam', 'Negeri', 'Sunan', 'Gunung', 'Djati', 'Bandung', 'irfanbahafuinsgdacid', 'Abstrak', 'Teknologi', 'komunikasi', 'dan', 'informasi', 'telah', 'berkembang', 'seiring', 'dengan', 'tuntutan', 'untuk', 'mendapatkan', 'informasi', 'yang', 'lebih', 'spesifik', 'dan', 'terukur', 'Data', 'yang', 'diolah', 'dijadikan', 'pendukung', 'untuk', 'mengambil', 'keputusan', 'dengan', 'memanfaatkan', 'basisdata', 'yang', 'dimiliki', 'untuk', 'menentukan', 'pola', 'asosiasi', 'jalur', 'masuk', 'calon', 'mahasiswa', 'terhadap', 'kelulusan', 'Pola', 'asosiasi', 'antara', 'data', 'mahasiswa', 'dan', 'data', 'kelulsan', 'mahasiswa', 'Pola', 'asosiasi', 

['edisi', 'juli', 'volume', 'ix', 'no', 'issn', 'analisa', 'pola', 'asosiasi', 'jalur', 'masuk', 'kelulus', 'mahasiswa', 'metode', 'foldgrowth', 'studi', 'fakultas', 'sains', 'teknologi', 'mohamad', 'irfan', 'jurus', 'teknik', 'informatika', 'fakultas', 'sains', 'teknologi', 'universitas', 'islam', 'negeri', 'sunan', 'gunung', 'djati', 'bandung', 'irfanbahafuinsgdacid', 'abstrak', 'teknologi', 'komunikasi', 'informasi', 'kembang', 'iring', 'tuntut', 'informasi', 'spesifik', 'ukur', 'data', 'olah', 'jadi', 'dukung', 'ambil', 'putus', 'manfaat', 'basisdata', 'milik', 'tentu', 'pola', 'asosiasi', 'jalur', 'masuk', 'calon', 'mahasiswa', 'kelulus', 'pola', 'asosiasi', 'data', 'mahasiswa', 'data', 'kelulsan', 'mahasiswa', 'pola', 'asosiasi', 'jenis', 'data', 'kait', 'variabel', 'data', 'mahasiswa', 'jalur', 'masuk', 'jurus', 'tingkat', 'kelulus', 'mahasiswa', 'studi', 'indeks', 'prestasi', 'kumulatif', 'ipk', 'teliti', 'cari', 'pola', 'asosiasi', 'jalur', 'masuk', 'kelulus', 'mahasiswa', 'me

['ANALISIS', 'BAHAN', 'AJAR', 'FISIKA', 'SMA', 'KELAS', 'XI', 'DI', 'KECAMATAN', 'INDRALAYA', 'UTARA', 'BERDASARKAN', 'KATEGORI', 'LITERASI', 'SAINS', 'Feni', 'Kurnia', 'Zulherman', 'Apit', 'Fathurohman', 'Alumini', 'Prodi', 'Pendidikan', 'Fisika', 'FKIP', 'Universitas', 'Sriwijaya', 'Dosen', 'Prodi', 'Pendidikan', 'Fisika', 'FKIP', 'Universitas', 'Sriwijaya', 'email', 'fenikurniaasryahoocom', 'Abtract', 'The', 'aim', 'of', 'this', 'research', 'is', 'to', 'analyze', 'whether', 'high', 'school', 'physics', 'teaching', 'materials', 'which', 'used', 'in', 'grade', 'XI', 'in', 'the', 'District', 'of', 'North', 'Indralaya', 'already', 'represent', 'the', 'Scientific', 'Literacy', 'themes', 'as', 'well', 'as', 'obtain', 'information', 'on', 'the', 'scope', 'of', 'science', 'of', 'teaching', 'materials', 'in', 'the', 'form', 'of', 'books', 'used', 'in', 'schools', 'in', 'the', 'district', 'of', 'North', 'Indralaya', 'This', 'research', 'was', 'done', 'using', 'physics', 'teaching', 'materials

...

['SEMINAR', 'NASIONAL', 'PENDIDIKAN', 'SAINS', 'melalui', 'Penelitian', 'Pengembangan', 'dalam', 'Menghadapi', 'Tantangan', 'Abad', 'Surakarta', 'Oktober', 'Prosiding', 'Seminar', 'Nasional', 'Pendidikan', 'Sains', 'SNPS', 'Putri', 'Agustina', 'Alanindra', 'Saputra', 'Prodi', 'PBiologi', 'FKIP', 'Universitas', 'Muhammadiyah', 'Surakarta', 'Sukoharjo', 'Prodi', 'P', 'Biologi', 'FKIP', 'Universitas', 'Sebelas', 'Maret', 'Surakarta', 'Email', 'Korespondensi', 'paumsacid', 'Abstrak', 'Anatomi', 'Tumbuhan', 'merupakan', 'salah', 'satu', 'matakuliah', 'wajib', 'bagi', 'mahasiswa', 'semester', 'Prodi', 'P', 'Biologi', 'FKIP', 'Universitas', 'Muhammadiyah', 'Surakarta', 'Matakuliah', 'ini', 'adalah', 'matakuliah', 'dasar', 'yang', 'memberikan', 'pengetahuan', 'konten', 'content', 'knowledge', 'dan', 'melatihkan', 'keterampilan', 'proses', 'sains', 'KPS', 'dasar', 'bagi', 'mahasiswa', 'calon', 'guru', 'Penelitian', 'ini', 'bertujuan', 'untuk', 'menganalisis', 'KPS', 'dasar', 'mahasiswa', 'calon

['seminar', 'nasional', 'didik', 'sains', 'teliti', 'kembang', 'hadap', 'tantang', 'abad', 'surakarta', 'oktober', 'prosiding', 'seminar', 'nasional', 'didik', 'sains', 'snps', 'putri', 'agustina', 'alanindra', 'saputra', 'prodi', 'pbiologi', 'fkip', 'universitas', 'muhammadiyah', 'surakarta', 'sukoharjo', 'prodi', 'p', 'biologi', 'fkip', 'universitas', 'sebelas', 'maret', 'surakarta', 'email', 'korespondensi', 'paumsacid', 'abstrak', 'anatomi', 'tumbuh', 'salah', 'matakuliah', 'wajib', 'mahasiswa', 'semester', 'prodi', 'p', 'biologi', 'fkip', 'universitas', 'muhammadiyah', 'surakarta', 'matakuliah', 'matakuliah', 'dasar', 'tahu', 'konten', 'content', 'knowledge', 'latih', 'terampil', 'proses', 'sains', 'kps', 'dasar', 'mahasiswa', 'calon', 'guru', 'teliti', 'tuju', 'analis', 'kps', 'dasar', 'mahasiswa', 'calon', 'guru', 'biologi', 'praktikum', 'anatomi', 'tumbuh', 'teliti', 'teliti', 'deskriptif', 'eksploratif', 'laksana', 'prodi', 'p', 'biologi', 'fkip', 'ums', 'februari', 'juli', 's

['i', 'ANALISIS', 'KETERAMPILAN', 'PROSES', 'SAINS', 'PESERTA', 'DIDIK', 'KELAS', 'XI', 'SEMESTER', 'II', 'MAN', 'TEMPEL', 'TAHUN', 'AJARAN', 'PADA', 'PEMBELAJARAN', 'KIMIA', 'DENGAN', 'MODEL', 'LEARNING', 'CYCLE', 'E', 'SKRIPSI', 'Diajukan', 'kepada', 'Fakultas', 'Matematika', 'dan', 'Ilmu', 'Pengetahuan', 'Alam', 'Universitas', 'Negeri', 'Yogyakarta', 'untuk', 'Memenuhi', 'Sebagian', 'Persyaratan', 'Guna', 'Memperoleh', 'Gelar', 'Sarjana', 'Pendidikan', 'Sains', 'Oleh', 'ASTRI', 'KURNIAWATI', 'NIM', 'JURUSAN', 'PENDIDIKAN', 'KIMIA', 'FAKULTAS', 'MATEMATIKA', 'DAN', 'ILMU', 'PENGETAHUAN', 'ALAM', 'UNIVERSITAS', 'NEGERI', 'YOGYAKARTA', 'ii', 'iii', 'iv', 'v', 'HALAMAN', 'MOTTO', 'Guru', 'yang', 'sukses', 'adalah', 'ketika', 'seorang', 'siswa', 'sudah', 'tidak', 'membutuhkan', 'guru', 'tersebut', 'Banyak', 'orang', 'mengira', 'bahwa', 'jalan', 'yang', 'menanjak', 'adalah', 'yang', 'paling', 'sulit', 'dilakukan', 'namun', 'sebenarnya', 'ketika', 'kita', 'sampai', 'dipuncak', 'ketinggian'

['i', 'analisis', 'terampil', 'proses', 'sains', 'serta', 'didik', 'kelas', 'xi', 'semester', 'ii', 'man', 'tempel', 'ajar', 'ajar', 'kimia', 'model', 'learning', 'cycle', 'e', 'skripsi', 'aju', 'fakultas', 'matematika', 'ilmu', 'tahu', 'alam', 'universitas', 'negeri', 'yogyakarta', 'penuh', 'syarat', 'oleh', 'gelar', 'sarjana', 'didik', 'sains', 'astri', 'kurniawati', 'nim', 'jurus', 'didik', 'kimia', 'fakultas', 'matematika', 'ilmu', 'tahu', 'alam', 'universitas', 'negeri', 'yogyakarta', 'ii', 'iii', 'iv', 'v', 'halaman', 'motto', 'guru', 'sukses', 'siswa', 'butuh', 'guru', 'orang', 'jalan', 'tanjak', 'sulit', 'puncak', 'tinggi', 'sana', 'tingkat', 'sulit', 'sungguh', 'puncak', 'hembus', 'angin', 'lindung', 'puncak', 'suluh', 'indah', 'takjub', 'nomor', 'derita', 'nomor', 'bong', 'chandra', 'vi', 'halaman', 'sembah', 'allah', 'swt', 'tiada', 'daya', 'upaya', 'kecuali', 'tolong', 'karya', 'hasil', 'usaha', 'semangat', 'kerja', 'keras', 'karya', 'keping', 'rangkai', 'cerita', 'dewasa',

PdfReadError: File has not been decrypted